## Text analytics for DEI stance & purchase intentions

### 1. Import 'combined_sentiment_annotations.csv'

In [1]:
# --- Imports ---
import pandas as pd
import numpy as np
import re
from sklearn.metrics import cohen_kappa_score
import os

In [2]:
# Define file path relative to the notebook or project root
csv_file_path = '../../data/annotate/complete/combined_sentiment_annotations.csv'

# Load the CSV file
df = pd.read_csv(csv_file_path)
print(f"Successfully loaded {csv_file_path}")
print(f"DataFrame shape: {df.shape}")
print("First 5 rows:")
df.head(6)

Successfully loaded ../../data/annotate/complete/combined_sentiment_annotations.csv
DataFrame shape: (1000, 25)
First 5 rows:


,company_name,post_date,id,parent_id,comment_text,comment_date,comment_type,reaction_count,before_DEI,has_DEI,...,full_text,relevance,stance_dei_reviewer_01,purchase_intention_reviewer_01,stance_dei_reviewer_02,purchase_intention_reviewer_02,stance_dei_average,purchase_intention_average,stance_dei_label,purchase_intention_label
0,Target,2/2/25 11:06,Y29tbWVudDoxMTcxMzM2NDkxMDE4NDA3XzExMzM1OTk4OT...,NaN,The audacity,2/7/25,initial,0,0,0,...,the audacity,0,0 (Neutral/Unclear towards DEI),0 (Neutral/Unclear/No PI),0 (Neutral/Unclear towards DEI),0 (Neutral/Unclear/No PI),0.0,0.0,0,0
1,Costco,1/13/25 11:00,Y29tbWVudDoxMDIwNjM1NjYzNDI3OTMwXzEyNjM0Njg2MD...,NaN,"Yummmmm.,",1/17/25,initial,0,1,1,...,"yummmmm.,",0,0 (Neutral/Unclear towards DEI),0 (Neutral/Unclear/No PI),0 (Neutral/Unclear towards DEI),0 (Neutral/Unclear/No PI),0.0,0.0,0,0
2,Costco,2/9/25 9:00,Y29tbWVudDoxMDM5NjIyNDU0ODYyNTg0XzM2OTUyNzExMj...,NaN,How much,2/16/25,initial,0,0,1,...,how much,0,0 (Neutral/Unclear towards DEI),0 (Neutral/Unclear/No PI),0 (Neutral/Unclear towards DEI),0 (Neutral/Unclear/No PI),0.0,0.0,0,0
3,Costco,2/16/25 9:00,Y29tbWVudDoxMDQ1NjAzNjA0MjY0NDY5XzkwODU0Mjg4ND...,NaN,That looks so good!!! It also looks like the l...,2/23/25,initial,1,0,1,...,that looks so good!!! it also looks like the l...,0,0 (Neutral/Unclear towards DEI),1 (Buy/Positive),0 (Neutral/Unclear towards DEI),1 (Buy/Positive),0.0,1.0,0,1
4,Delta,2/13/25 9:30,Y29tbWVudDoxMDIxMzAxOTAwMDE4NzIyXzIwMDQ2MzM4Mj...,NaN,I have a friend who wanted to be a pilot for D...,2/15/25,initial,0,0,1,...,i have a friend who wanted to be a pilot for d...,0,0 (Neutral/Unclear towards DEI),0 (Neutral/Unclear/No PI),0 (Neutral/Unclear towards DEI),0 (Neutral/Unclear/No PI),0.0,0.0,0,0
5,Costco,2/11/25 9:00,Y29tbWVudDoxMDQxNzYyOTI3OTgxODcwXzExNTU4NjE1OT...,Y29tbWVudDoxMDQxNzYyOTI3OTgxODcwXzgyODIwNTI2Mj...,another person who doesn’t understand DEI,2/16/25,reply,19,0,1,...,how much money are your executives getting to ...,1,-1 (Anti-DEI),0 (Neutral/Unclear/No PI),-1 (Anti-DEI),0 (Neutral/Unclear/No PI),-1.0,0.0,-1,0


In [3]:
# Function to extract numeric score from label string (handles potential errors)
def extract_score(label):
    if pd.isna(label) or not isinstance(label, str):
        return np.nan
    # Match integers (positive, negative, or zero) at the beginning of the string
    match = re.match(r"^(-?\d+)", label)
    if match:
        return int(match.group(1))
    return np.nan # Return NaN if no number is found at the beginning

reviewer_cols = [
    'stance_dei_reviewer_01', 'purchase_intention_reviewer_01',
    'stance_dei_reviewer_02', 'purchase_intention_reviewer_02'
    ]

# Check if columns exist before processing
missing_cols = [col for col in reviewer_cols if col not in df.columns]

df['stance_dei_score_r1'] = df['stance_dei_reviewer_01'].apply(extract_score)
df['pi_score_r1'] = df['purchase_intention_reviewer_01'].apply(extract_score)
df['stance_dei_score_r2'] = df['stance_dei_reviewer_02'].apply(extract_score)
df['pi_score_r2'] = df['purchase_intention_reviewer_02'].apply(extract_score)

In [4]:
final_label_cols = ['stance_dei_label', 'purchase_intention_label']

print("Counts for final 'stance_dei_label':")
dei_counts = df['stance_dei_label'].value_counts(dropna=False) # Include NaNs if any
print(dei_counts)

print("\n")

print("Counts for final 'purchase_intention_label':")
pi_counts = df['purchase_intention_label'].value_counts(dropna=False) # Include NaNs if any
print(pi_counts)

Counts for final 'stance_dei_label':
stance_dei_label
 0    784
 1    126
-1     90
Name: count, dtype: int64


Counts for final 'purchase_intention_label':
purchase_intention_label
 0    762
-1    127
 1    111
Name: count, dtype: int64


### 2. Calculate Cohen's Kappa for dual-coded annotations

In [5]:
df_kappa_dei = df[['stance_dei_score_r1', 'stance_dei_score_r2']].dropna()
print(f"Calculating Kappa for DEI Stance using {len(df_kappa_dei)} complete pairs of ratings.")

kappa_dei = cohen_kappa_score(df_kappa_dei['stance_dei_score_r1'], df_kappa_dei['stance_dei_score_r2'])
print(f"Cohen's Kappa for DEI Stance: {kappa_dei:.4f}")

print("\n")

df_kappa_pi = df[['pi_score_r1', 'pi_score_r2']].dropna()
print(f"Calculating Kappa for Purchase Intention using {len(df_kappa_pi)} complete pairs of ratings.")

kappa_pi = cohen_kappa_score(df_kappa_pi['pi_score_r1'], df_kappa_pi['pi_score_r2'])
print(f"Cohen's Kappa for Purchase Intention: {kappa_pi:.4f}")

Calculating Kappa for DEI Stance using 1000 complete pairs of ratings.
Cohen's Kappa for DEI Stance: 0.8173


Calculating Kappa for Purchase Intention using 1000 complete pairs of ratings.
Cohen's Kappa for Purchase Intention: 0.7385


### 3. Prepare Labels for Imbalance Handling

In [6]:
# Map labels from [-1, 0, 1] to [0, 1, 2] for compatibility with loss functions
y_stance_original = df['stance_dei_label'].astype(int).values
y_pi_original = df['purchase_intention_label'].astype(int).values

# Stance: -1 (Anti) -> 0, 0 (Neutral) -> 1, 1 (Pro) -> 2
y_stance_idx = y_stance_original + 1

# PI: -1 (Boycott) -> 0, 0 (Neutral) -> 1, 1 (Buy) -> 2
y_pi_idx = y_pi_original + 1

print(f"Example DEI stance indices: {y_stance_idx[:10]}")
print(f"Example PI indices: {y_pi_idx[:10]}")

# Define class names for reporting
stance_class_names = ["anti", "neutral", "pro"]
pi_class_names = ["boycott", "neutral", "buy"]

print("\n")

# Ensure classes are correctly identified (0, 1, 2)
stance_classes = np.unique(y_stance_idx) # Should be [0, 1, 2]
print(f"Unique stance classes found for weighting: {stance_classes}")
pi_classes = np.unique(y_pi_idx) # Should be [0, 1, 2]      
print(f"Unique PI classes found for weighting: {pi_classes}")

Example DEI stance indices: [1 1 1 1 1 0 0 1 1 2]
Example PI indices: [1 1 1 2 1 1 0 1 0 0]


Unique stance classes found for weighting: [0 1 2]
Unique PI classes found for weighting: [0 1 2]


In [7]:
# Let's define our PyTorch Device (I am using MPS for Apple Silicon on Macbook M3 Pro)
import torch

if torch.backends.mps.is_available():
    # Check if MPS is available
    device = torch.device("mps")
    print("MPS backend is available. Using MPS device.")
elif not torch.backends.mps.is_built():
    # Check if MPS is built (required for is_available to be True)
    # This case is unlikely if is_available() is False, but good to be explicit
    device = torch.device("cpu")
    print("MPS not available because the current PyTorch install was not built with MPS enabled.")
else:
    # MPS is built but not available (e.g., OS version issue, though unlikely on modern macOS)
    device = torch.device("cpu")
    print("MPS not available. Using CPU device.")

print(f"Selected device: {device}")

MPS backend is available. Using MPS device.
Selected device: mps


In [8]:
# Now we are going to do some weighted random sampling 
from torch.utils.data import DataLoader, WeightedRandomSampler

# Compute sample weights for Stance DEI (one weight per example)
stance_label_counts = {label: count for label, count in zip(*np.unique(y_stance_idx, return_counts=True))}
print(f"Stance Label Counts: {stance_label_counts}")

# Avoid division by zero if a class somehow has 0 counts (shouldn't happen with unique)
stance_sample_weights = [1.0 / stance_label_counts[label] if stance_label_counts[label] > 0 else 0 for label in y_stance_idx]

# Create the sampler
stance_sampler = WeightedRandomSampler(stance_sample_weights, num_samples=len(stance_sample_weights), replacement=True)
print("Created WeightedRandomSampler for Stance DEI.")

print("\n")

# Compute sample weights for Purchase Intention
pi_label_counts = {label: count for label, count in zip(*np.unique(y_pi_idx, return_counts=True))}
print(f"PI Label Counts: {pi_label_counts}")

pi_sample_weights = [1.0 / pi_label_counts[label] if pi_label_counts[label] > 0 else 0 for label in y_pi_idx]

# Create the sampler
pi_sampler = WeightedRandomSampler(pi_sample_weights, num_samples=len(pi_sample_weights), replacement=True)
print("Created WeightedRandomSampler for Purchase Intention.")

Stance Label Counts: {np.int64(0): np.int64(90), np.int64(1): np.int64(784), np.int64(2): np.int64(126)}
Created WeightedRandomSampler for Stance DEI.


PI Label Counts: {np.int64(0): np.int64(127), np.int64(1): np.int64(762), np.int64(2): np.int64(111)}
Created WeightedRandomSampler for Purchase Intention.


### 4. Format full_text input for DeBERTa-LoRa

In [9]:
from torch.utils.data import Dataset
from transformers import AutoTokenizer
import numpy as np

# Let's define a lambda function to handle the splitting and NaN cases inline
split_lambda = lambda text: [] if pd.isna(text) else text.split(' → ')

# Apply the lambda function to create a new column with the list of segments
df['text_segments'] = df['full_text'].apply(split_lambda)

# Show examples
print(df[df['full_text'].str.contains(" → ", na=False)][['full_text', 'text_segments']].head().to_markdown(index=False))

| full_text                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         | text_segments                                                                                                                                                                                                                                                     

In [10]:
import torch
from torch.utils.data import Dataset
from transformers import DebertaV2Tokenizer

MODEL_NAME = "microsoft/deberta-v3-large"
tokenizer  = DebertaV2Tokenizer.from_pretrained(MODEL_NAME, use_fast=False)

special_tokens = {"additional_special_tokens": ["<CONTEXT>", "</CONTEXT>",
                                                "<REPLY>", "</REPLY>"]}
tokenizer.add_special_tokens(special_tokens)

print("sep_token →", tokenizer.sep_token)

df["stance_label_idx"] = y_stance_idx # -1,0,1 → 0,1,2

df["pi_label_idx"] = y_pi_idx

class SentimentDataset(Dataset):          
    def __init__(self, df, tok, max_len):
        self.segs  = df["text_segments"].tolist()
        self.y_s   = df["stance_label_idx"].tolist()
        self.y_p   = df["pi_label_idx"].tolist()
        self.tok   = tok; self.max_len = max_len
    def __len__(self):  return len(self.segs)
    def __getitem__(self, i):
        segs = self.segs[i]
        txt  = f"<REPLY> {segs[0]} </REPLY>" if len(segs)==1 else \
               f"<CONTEXT> {' </CONTEXT> <CONTEXT> '.join(segs[:-1])} </CONTEXT> " \
               f"<REPLY> {segs[-1]} </REPLY>"
        enc  = self.tok(txt, add_special_tokens=False, max_length=self.max_len,
                        padding=False, truncation=True,
                        return_attention_mask=True, return_tensors="pt")
        return { "input_ids":      enc["input_ids"].squeeze(),
                 "attention_mask": enc["attention_mask"].squeeze(),
                 "stance_label":   torch.tensor(self.y_s[i]),
                 "pi_label":       torch.tensor(self.y_p[i]) }

MAX_LEN = 512
preview_ds = SentimentDataset(df, tokenizer, MAX_LEN)
print("Decoded preview:", tokenizer.decode(preview_ds[0]["input_ids"],
                                           skip_special_tokens=False))

def join_segments(segs):
    """
    Re‑assemble the list returned by `split_lambda` into the exact text
    string that will be fed to the tokenizer later on.
    """
    if not segs:                         # blank row guard
        return ""
    if len(segs) == 1:                   # only a reply (no parents)
        return f"<REPLY> {segs[0]} </REPLY>"
    context = " </CONTEXT> <CONTEXT> ".join(segs[:-1])
    reply   = segs[-1]
    return f"<CONTEXT> {context} </CONTEXT> <REPLY> {reply} </REPLY>"

df["joined_text"] = df["text_segments"].apply(join_segments)

sep_token → [SEP]
Decoded preview: <REPLY> the audacity </REPLY>


### 5. LLM Classification

In [ ]:
# ═══════════════════════════════════════════════════
# 0)  ENV  ─ set before torch / transformers
# ═══════════════════════════════════════════════════
import os, warnings, random, gc, numpy as np
os.environ.pop("PYTORCH_MPS_HIGH_WATERMARK_RATIO", None)
os.environ["PYTORCH_MPS_HIGH_WATERMARK_RATIO"] = "0.9"

SEED = 42
random.seed(SEED); np.random.seed(SEED)

# ═══════════════════════════════════════════════════
# 1)  LIBS
# ═══════════════════════════════════════════════════
import torch, torch.nn as nn, torch.nn.functional as F
from torch.utils.data import DataLoader, WeightedRandomSampler, random_split
from torch.utils.tensorboard import SummaryWriter
from torch.optim import AdamW
from transformers import (AutoTokenizer, AutoModel,
                          get_cosine_schedule_with_warmup)
from sklearn.metrics import f1_score, confusion_matrix
from sklearn.utils.class_weight import compute_class_weight
from tqdm.auto import tqdm
from peft import LoraConfig, get_peft_model, TaskType, PeftModel

warnings.filterwarnings("ignore")
torch.set_float32_matmul_precision("medium")

device = (
    torch.device("mps")  if torch.backends.mps.is_available() else
    torch.device("cuda") if torch.cuda.is_available()        else
    torch.device("cpu")
)
print("Running on ➜", device)

# ═══════════════════════════════════════════════════
# 2)  HYPER‑PARAMS   (updated per instructions)
# ═══════════════════════════════════════════════════
MODEL_NAME   = "microsoft/deberta-v3-large"
EPOCHS       = 30
PATIENCE     = 6
BATCH        = 16        # micro‑batch
ACC_STEPS    = 2         # 16×2 ⇒ eff‑batch = 32
LR_BACKBONE  = 3e-6
LR_HEADS     = 3e-5      # 10 × faster
MAX_NORM     = 5.0
WARMUP_FRAC  = .1
GAMMA        = 2.0       # focal‑loss γ
NUM_LABELS   = 3
LABELS       = np.arange(NUM_LABELS)

# ═══════════════════════════════════════════════════
# 3)  TOKENISER  (resize‑tokens only)
# ═══════════════════════════════════════════════════
tok = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=False)
tok.add_special_tokens(
    {"additional_special_tokens": ["<CONTEXT>", "</CONTEXT>",
                                   "<REPLY>", "</REPLY>"]})

# ═══════════════════════════════════════════════════
# 4)  DATA  – load cache, pin once on GPU
# ═══════════════════════════════════════════════════
cache = np.load("cached_encodings.npz", mmap_mode="r")
IDS   = torch.from_numpy(cache["ids"   ]).to(device, non_blocking=True)
MASK  = torch.from_numpy(cache["mask"  ]).to(device, non_blocking=True)
Y_S   = torch.from_numpy(cache["stance"]).to(device)
Y_P   = torch.from_numpy(cache["pi"    ]).to(device)

class GPUSliceSet(torch.utils.data.Dataset):
    def __init__(self, ids, mask, ys, yp):
        self.ids, self.mask, self.ys, self.yp = ids, mask, ys, yp
    def __len__(self): return self.ids.size(0)
    def __getitem__(self, i):
        return {"input_ids":      self.ids [i],
                "attention_mask": self.mask[i],
                "stance_label":   self.ys  [i],
                "pi_label":       self.yp  [i]}

full_ds           = GPUSliceSet(IDS, MASK, Y_S, Y_P)
train_ds, val_ds  = random_split(full_ds, [int(.8*len(full_ds)),
                                           len(full_ds)-int(.8*len(full_ds))])
train_idx_gpu = torch.tensor(train_ds.indices, device=device)

# ═══════════════════════════════════════════════════
# 5)  SAMPLER  (balanced – new shuffle each epoch)
# ═══════════════════════════════════════════════════
def make_sampler(epoch_seed: int) -> WeightedRandomSampler:
    torch.manual_seed(epoch_seed)
    lbl_cpu = Y_S[train_idx_gpu].to("cpu", dtype=torch.long)
    cnt     = torch.bincount(lbl_cpu, minlength=NUM_LABELS).float()
    w       = 1.0 / cnt.clamp(min=1)
    weights = w[lbl_cpu]
    return WeightedRandomSampler(weights, len(weights), replacement=True)

# ═══════════════════════════════════════════════════
# 6)  MODEL  – LoRA r = 32, α = 64, + out_proj, bf16
# ═══════════════════════════════════════════════════
def build_backbone():
    base = AutoModel.from_pretrained(MODEL_NAME, low_cpu_mem_usage=True)
    base.gradient_checkpointing_enable()
    cfg  = LoraConfig(
            task_type      = TaskType.FEATURE_EXTRACTION,
            r              = 32,
            lora_alpha     = 64,
            lora_dropout   = .05,
            target_modules = ["query_proj", "value_proj", "out_proj"])
    base = get_peft_model(base, cfg)
    base.resize_token_embeddings(len(tok))
    return base

class MultiHead(nn.Module):
    def __init__(self):
        super().__init__()
        self.backbone = build_backbone()
        h = self.backbone.config.hidden_size
        self.head_s  = nn.Linear(h, NUM_LABELS)
        self.head_pi = nn.Linear(h, NUM_LABELS)
    def forward(self, ids, mask):
        with torch.autocast(device.type, torch.bfloat16):
            cls = self.backbone(ids, attention_mask=mask).last_hidden_state[:, 0]
        return self.head_s(cls), self.head_pi(cls)

model = MultiHead().to(device)
print(); model.backbone.print_trainable_parameters()

# separate param groups
backbone_params = list(model.backbone.parameters())
head_params     = list(model.head_s.parameters()) + list(model.head_pi.parameters())

# ═══════════════════════════════════════════════════
# 7)  FOCAL‑LOSSES  (γ = 2)
# ═══════════════════════════════════════════════════
def focal_loss(alpha, gamma):
    class _F(nn.Module):
        def __init__(self): super().__init__(); self.a=alpha
        def forward(self, logit, tgt):
            ce = F.cross_entropy(logit, tgt, weight=self.a, reduction="none")
            return ((1.0 - torch.exp(-ce))**gamma * ce).mean()
    return _F()

w_s = torch.tensor(compute_class_weight("balanced",
                                        classes=LABELS,
                                        y=Y_S[train_idx_gpu].cpu().numpy()),
                   dtype=torch.float, device=device)
w_p = torch.tensor(compute_class_weight("balanced",
                                        classes=LABELS,
                                        y=Y_P[train_idx_gpu].cpu().numpy()),
                   dtype=torch.float, device=device)

loss_s = focal_loss(w_s, GAMMA)
loss_p = focal_loss(w_p, GAMMA)

# ═══════════════════════════════════════════════════
# 8)  OPTIMISER & SCHEDULER  (two‑tier LR)
# ═══════════════════════════════════════════════════
optim = AdamW([
        {"params": backbone_params, "lr": LR_BACKBONE},
        {"params": head_params,     "lr": LR_HEADS    }],
        betas=(0.9,0.999), weight_decay=0.01)

steps_per_epoch = len(train_ds) // BATCH
total_steps     = steps_per_epoch * EPOCHS
sched = get_cosine_schedule_with_warmup(
    optim, int(WARMUP_FRAC * total_steps), total_steps, num_cycles=2)

# ═══════════════════════════════════════════════════
# 9)  TRAIN / EVAL
# ═══════════════════════════════════════════════════
def train_one_epoch(ep):
    sampler = make_sampler(ep)
    loader  = DataLoader(train_ds, batch_size=BATCH,
                         sampler=sampler, drop_last=True)
    model.train(); running=0.
    bar = tqdm(loader, desc="train", leave=False)
    for step, b in enumerate(bar, 1):
        ids,mask = b["input_ids"], b["attention_mask"]
        ys,yp    = b["stance_label"], b["pi_label"]
        s,p      = model(ids,mask)
        loss     = (loss_s(s,ys)+loss_p(p,yp))/ACC_STEPS
        loss.backward()
        if step % ACC_STEPS == 0:
            torch.nn.utils.clip_grad_norm_(model.parameters(), MAX_NORM)
            optim.step(); sched.step(); optim.zero_grad(set_to_none=True)
        running += loss.item()*ACC_STEPS
        bar.set_postfix(loss = running/step)
    return running/step

@torch.no_grad()
def evaluate():
    loader = DataLoader(val_ds, batch_size=BATCH)
    model.eval(); ps_s,ps_p,ls_s,ls_p=[],[],[],[]
    for b in loader:
        s,p = model(b["input_ids"], b["attention_mask"])
        ps_s.append(s.argmax(-1).cpu()); ls_s.append(b["stance_label"].cpu())
        ps_p.append(p.argmax(-1).cpu()); ls_p.append(b["pi_label"].cpu())
    ps_s,ls_s = torch.cat(ps_s),torch.cat(ls_s)
    ps_p,ls_p = torch.cat(ps_p),torch.cat(ls_p)

    cm_s = confusion_matrix(ls_s, ps_s, labels=LABELS)
    cm_p = confusion_matrix(ls_p, ps_p, labels=LABELS)
    f1_s = f1_score(ls_s, ps_s, average="macro")
    f1_p = f1_score(ls_p, ps_p, average="macro")

    print("\nConfusion (stance)\n", cm_s,
          "\nConfusion (PI)\n", cm_p,
          f"\nstance F1 {f1_s:.3f}   pi F1 {f1_p:.3f}")

    return f1_s+f1_p, f1_s, f1_p

def clear_mps():
    if device.type=="mps": torch.mps.empty_cache()
    gc.collect()

# ═══════════════════════════════════════════════════
# 10)  LOOP
# ═══════════════════════════════════════════════════
writer,best,wait = SummaryWriter(), -1e9, 0
for ep in range(1,EPOCHS+1):
    print(f"\n🟢 Epoch {ep}/{EPOCHS}")
    tr_loss = train_one_epoch(ep)
    score,f1_s,f1_p = evaluate()

    writer.add_scalar("loss/train", tr_loss, ep)
    writer.add_scalars("val", {"stance_f1":f1_s, "pi_f1":f1_p}, ep); writer.flush()

    if score > best: best,wait = score,0
    else:
        wait += 1
        if wait >= PATIENCE:
            print("🔴 early stop – no improvement"); break
    clear_mps()

writer.close(); clear_mps()


Running on ➜ mps

trainable params: 3,145,728 || all params: 437,060,608 || trainable%: 0.7197

🟢 Epoch 1/30


train:   0%|          | 0/50 [00:00<?, ?it/s]


Confusion (stance)
 [[  0  14   1]
 [  2 159   0]
 [  0  24   0]] 
Confusion (PI)
 [[ 19   0   1]
 [157   0   1]
 [ 22   0   0]] 
stance F1 0.296   pi F1 0.058

🟢 Epoch 2/30


train:   0%|          | 0/50 [00:00<?, ?it/s]


Confusion (stance)
 [[  0  14   1]
 [  2 158   1]
 [  0  24   0]] 
Confusion (PI)
 [[ 19   0   1]
 [156   0   2]
 [ 22   0   0]] 
stance F1 0.295   pi F1 0.058

🟢 Epoch 3/30


...


Confusion (stance)
 [[ 14   0   1]
 [158   0   3]
 [ 24   0   0]] 
Confusion (PI)
 [[ 19   0   1]
 [157   0   1]
 [ 22   0   0]] 
stance F1 0.044   pi F1 0.058

🟢 Epoch 6/30


train:   0%|          | 0/50 [00:00<?, ?it/s]


Confusion (stance)
 [[ 14   0   1]
 [159   0   2]
 [ 24   0   0]] 
Confusion (PI)
 [[ 19   0   1]
 [157   0   1]
 [ 22   0   0]] 
stance F1 0.044   pi F1 0.058

🟢 Epoch 7/30


train:   0%|          | 0/50 [00:00<?, ?it/s]


Confusion (stance)
 [[ 14   0   1]
 [159   0   2]
 [ 24   0   0]] 
Confusion (PI)
 [[ 20   0   0]
 [157   0   1]
 [ 22   0   0]] 
stance F1 0.044   pi F1 0.061
🔴 early stop – no improvement
